<a href="https://colab.research.google.com/github/AvantiShri/oceanography_colab_notebooks/blob/master/for_rian/ArchetypeAnalysis_GP15_WaterMasses.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install py_pcha
!pip install gsw

Grab the data

In [2]:
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1O869bUhoTrPCW4hDaSVswLiQ7vzhzIlP' -O names_added_GP15OMPA_33RR20180918_only_gs_rosette_clean1_hy1.csv

--2020-08-28 17:33:10--  https://docs.google.com/uc?export=download&id=1O869bUhoTrPCW4hDaSVswLiQ7vzhzIlP
Resolving docs.google.com (docs.google.com)... 172.217.204.100, 172.217.204.139, 172.217.204.101, ...
Connecting to docs.google.com (docs.google.com)|172.217.204.100|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-0s-a4-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/1h4gt60n9t3qhu96slaltgqjapsm30g1/1598635950000/06203730782251856755/*/1O869bUhoTrPCW4hDaSVswLiQ7vzhzIlP?e=download [following]
--2020-08-28 17:33:11--  https://doc-0s-a4-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/1h4gt60n9t3qhu96slaltgqjapsm30g1/1598635950000/06203730782251856755/*/1O869bUhoTrPCW4hDaSVswLiQ7vzhzIlP?e=download
Resolving doc-0s-a4-docs.googleusercontent.com (doc-0s-a4-docs.googleusercontent.com)... 172.217.203.132, 2607:f8b0:400c:c07::84
Connecting to doc-0s-a4-docs.googleusercontent.com (doc-0s

In [3]:
from matplotlib import pyplot as plt
import numpy as np
import pandas
import gsw

Read in the data frame and rename the columns

In [4]:
#Easy remapping of the column names
colnames_map = {'Station number':"stnnbr",
            'GEOTRACES ID':"geotrc_ID",
            'latitude (degrees)':"lat",
            'longitude (degrees)':"lon",
            'depth (m)':"depth",
            'pressure (dbar)':"pres",
            'temperature(degrees C)':"t",
            'salinity (psu)':"SP",
            'oxygen (umol/kg)':"O2",
            'silicate (umol/kg)':"Si",
            'nitrate (umol/kg)':"NO3",
            'phosphate (umol/kg)':"PO4",
            'potential density':"sig0",
            'PO (umol/kg)':"PO",
            }

ROUNDING_PRECISION = 4


#For some reason, altair chokes when provided data frames with some
# of the original column names. So I am remapping the column names.
def remap_colnames(df, colnames_map):
  remapnames_df = pandas.DataFrame(dict([
      (new_col, np.array(df[orig_col]))
      for new_col,orig_col in colnames_map.items()]))
  return remapnames_df

df = pandas.read_csv("names_added_GP15OMPA_33RR20180918_only_gs_rosette_clean1_hy1.csv", na_values = -999)
df.columns
#, sep='delimiter',header=None
remapnames_df = remap_colnames(df=df, colnames_map=colnames_map)
#create a column for calculated variables
remapnames_df['NO'] = remapnames_df['oxygen (umol/kg)'] + (remapnames_df['nitrate (umol/kg)']*9.68)
remapnames_df['pt'] = np.round(
    gsw.pt_from_t(remapnames_df['salinity (psu)'],
                  remapnames_df['temperature(degrees C)'],
                  remapnames_df['pressure (dbar)'],
                  remapnames_df['potential density']),
   decimals=ROUNDING_PRECISION)

Prepare the features that define the convex hull

In [5]:
import sklearn.impute

#the columns to use for defining the convex hull
COLUMNS_TO_COMPARE = [
            'pt',
            'salinity (psu)',
            'silicate (umol/kg)',
            'potential density',
            'PO (umol/kg)',
            'NO']

#Let's standardize each column by subtracting mean and
# dividing by standard deviation. Call it a 'features' dataframe
features_df = pandas.DataFrame()
#keep track of mean and std in order to do inverse transform
colname_to_mean = {}
colname_to_std = {} 
for colname in COLUMNS_TO_COMPARE:
  vals = np.array(remapnames_df[colname])
  #use nanmean and nanstd to ignore nan values for now
  mean = np.nanmean(vals)
  std = np.nanstd(vals)
  colname_to_mean[colname] = mean
  colname_to_std[colname] = std
  features_df['zscore_'+colname] = (vals-mean)/std

#we impute nan values using KNNImputer
features_df = pandas.DataFrame(data=sklearn.impute.KNNImputer(
    missing_values=np.nan, n_neighbors=5,
    weights='distance').fit_transform(features_df),
    columns=features_df.columns)

#prepare a 'features' matrix for each point
features = np.array([np.array(features_df["zscore_"+col])
                     for col in COLUMNS_TO_COMPARE]).transpose((1,0))

TRANSFORM_MEANS = np.array([colname_to_mean[colname]
                           for colname in COLUMNS_TO_COMPARE])
TRANSFORM_STDS = np.array([colname_to_std[colname]
                           for colname in COLUMNS_TO_COMPARE])
#map features back to original space
def map_features_back(features):
  return features*TRANSFORM_STDS[None,:] + TRANSFORM_MEANS[None,:]

In [6]:
#Defining some helper functions for storing the archetype info and making
# altair-friendly dataframes

#get the archetypes in the original feature space
def get_archetypes_df(archetype_features):
  #map features back to the original space
  archetype_orig_features = map_features_back(archetype_features)
  archetype_df = pandas.DataFrame()
  for feature_idx, colname in enumerate(COLUMNS_TO_COMPARE):
    archetype_df[colname] = archetype_orig_features[:,feature_idx]
  archetype_df["archetype"] = ["archetype_"+str(i) for i in
                                   range(archetype_features.shape[0])]
  return archetype_df


#save the proportions of each archetype
def save_archetype_compositions(archetype_compositions, df):
  for archetype_num in range(archetype_compositions.shape[1]):
    df['archetype_'+str(archetype_num)] = np.round(
      archetype_compositions[:, archetype_num], decimals=ROUNDING_PRECISION)


#save the reconstructed features in the original feature space
def save_reconstructed_features(reconstructed_features, df):
  #map features back to the original space
  reconstructed_orig_features = map_features_back(reconstructed_features)
  for feature_idx, colname in enumerate(COLUMNS_TO_COMPARE):
    df["reconstructed_"+colname] = np.round(
        reconstructed_orig_features[:,feature_idx], decimals=ROUNDING_PRECISION)


#Create a df where there is a feature called 'composition' and a feature
# called 'archetype', and 'composition' indicates the proportion of the
# archetype
def get_archetype_color_df(orig_df):
  feature_names = list(remapnames_df.columns)


In [7]:
#Let's identify archetypes
from py_pcha import PCHA
import matplotlib


NUM_ENDMEMBERS = 3
XC, S, C, SSE, varexpl = PCHA(X=features.T, noc=NUM_ENDMEMBERS, verbose=False)
print("variance explained:",varexpl)

#get the archetype_dataframe containing original archetype features
archetypes_df = get_archetypes_df(archetype_features=np.array(XC).transpose())

archetype_compositions = np.array(S).transpose()
save_archetype_compositions(
    archetype_compositions=np.array(S).transpose(),
    df=remapnames_df)

save_reconstructed_features(
    reconstructed_features=np.array(XC*S).transpose(),
    df=remapnames_df)

##Old code for computing a color when there are only 3 archetypes
remapnames_df['archetypecolors'] = [
  matplotlib.colors.rgb2hex((a1,a2,a3))
  for (a1,a2,a3) in zip(archetype_compositions[:,0],
                        archetype_compositions[:,1],
                        archetype_compositions[:,2])
]

variance explained: 0.9435972836676995


In [8]:
print("The archetypes are:")
archetypes_df

The archetypes are:


,pt,salinity (psu),silicate (umol/kg),potential density,PO (umol/kg),NO,archetype
0,13.003743,34.867349,6.293728,26.293172,332.445374,297.149657,archetype_0
1,1.146451,34.682094,149.238437,27.795857,538.504084,493.343819,archetype_1
2,4.681186,32.840416,30.105712,26.003215,556.691016,491.916683,archetype_2


View altair interactive visualizations


In [27]:
import altair as alt

OBS_DF = remapnames_df #data frame for observations
ARCHETYPES_DF = archetypes_df #data frame for archetypes
INTERVAL_SELECTION = alt.selection_interval()
CHART_WIDTH=400
CHART_HEIGHT=200
TSNE_HEIGHTFRAC=0.4
TSNE_WIDTHFRAC=0.2
FONTSIZE=10
PADDING_GUESS=45 #additional padding to subtract off


#convenience functions to turn off default altair behaviour of including
# zero in the axis even if no points are at 0
def nozero_xaxis(field_name):
  return alt.X(field_name, scale=alt.Scale(zero=False))

def nozero_yaxis(field_name, domain=None):
  if (domain is None):
    return alt.Y(field_name, scale=alt.Scale(zero=False))
  else:
    return alt.Y(field_name, scale=alt.Scale(zero=False, domain=domain))


#define the color property that will be shared for the scatterplots/legend
color = alt.condition(INTERVAL_SELECTION, 'archetypecolors', alt.value('lightgray'),
                      scale=None)
#color = alt.condition(COMPOSED_SELECTION, 'squared_errors', alt.value('lightgray'))

tooltip_columns = (
    ['latitude (degrees)', 'longitude (degrees)', 'depth (m)']
   +['archetype_'+str(i) for i in range(NUM_ENDMEMBERS)]
   +[prefix+x for x in COLUMNS_TO_COMPARE for prefix in ['', 'reconstructed_']])

#base chart for displaying the observed points
obs_basechart = alt.Chart(OBS_DF).mark_point(opacity=0.3).encode(
  color=color,
  tooltip=tooltip_columns
).properties(width=CHART_WIDTH,
             height=CHART_HEIGHT).add_selection(INTERVAL_SELECTION)

#base chart for displaying the archetypes
archetype_basechart = alt.Chart(ARCHETYPES_DF).mark_point(opacity=1).encode(
    color=alt.Color("archetype", scale=alt.Scale(scheme="category10") ) ).properties(
        width=CHART_WIDTH, height=CHART_HEIGHT)

def create_scatter_obsonly(xaxis, yaxis):
  diagonal_df = pandas.DataFrame({'x': OBS_DF[xaxis], 'y': OBS_DF[xaxis]})
  return (obs_basechart.encode(nozero_xaxis(xaxis), nozero_yaxis(yaxis))
          + alt.Chart(diagonal_df).mark_line().encode(x='x', y='y', color=alt.value('black') ))

def create_scatter(xaxis, yaxis):
  return obs_basechart.encode(nozero_xaxis(xaxis), nozero_yaxis(yaxis)) + archetype_basechart.encode(nozero_xaxis(xaxis), nozero_yaxis(yaxis))
  #return archetype_basechart.encode(nozero_xaxis(xaxis), nozero_yaxis(yaxis))


#compose the whole layout
alt.vconcat(

(create_scatter_obsonly('pt', 'reconstructed_pt')
| create_scatter_obsonly('salinity (psu)', 'reconstructed_salinity (psu)')
| create_scatter_obsonly('silicate (umol/kg)', 'reconstructed_silicate (umol/kg)')),

(create_scatter_obsonly('potential density', 'reconstructed_potential density')
| create_scatter_obsonly('PO (umol/kg)', 'reconstructed_PO (umol/kg)')
| create_scatter_obsonly('NO', 'reconstructed_NO')),

(create_scatter('pt','salinity (psu)')
| create_scatter('salinity (psu)', 'silicate (umol/kg)')
| create_scatter('pt','silicate (umol/kg)')),

(create_scatter('pt','NO')
| create_scatter('salinity (psu)', 'NO')
| create_scatter('silicate (umol/kg)','NO')),
 
(create_scatter('pt','PO (umol/kg)')
| create_scatter('salinity (psu)', 'PO (umol/kg)')
| create_scatter('NO', 'PO (umol/kg)')),

(obs_basechart.encode(nozero_xaxis('latitude (degrees)'),
             nozero_yaxis('depth (m)', domain=(6000, 0))))

).configure_axis(labelFontSize=FONTSIZE,
                 titleFontSize=FONTSIZE).properties(padding=0, spacing=0)
# the padding/spacing doesn't propagate to subcharts propertly


alt.VConcatChart(...)

In [13]:
remapnames_df.columns

Index(['Station number', 'GEOTRACES ID', 'latitude (degrees)',
       'longitude (degrees)', 'depth (m)', 'pressure (dbar)',
       'temperature(degrees C)', 'salinity (psu)', 'oxygen (umol/kg)',
       'silicate (umol/kg)', 'nitrate (umol/kg)', 'phosphate (umol/kg)',
       'potential density', 'PO (umol/kg)', 'NO', 'pt', 'archetype_0',
       'archetype_1', 'archetype_2', 'reconstructed_pt',
       'reconstructed_salinity (psu)', 'reconstructed_silicate (umol/kg)',
       'reconstructed_potential density', 'reconstructed_PO (umol/kg)',
       'reconstructed_NO', 'archetypecolors'],
      dtype='object')